In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
import re
import plotly.express as px
from ipywidgets import interactive
from IPython.display import display, IFrame
from bs4 import BeautifulSoup
from pathlib import Path
from io import StringIO

In [2]:
# Show many outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def get_data_list(data_folder, extension=False):
    github_url = f"https://github.com/munizrodrigo/datathons-time6-dados/tree/main/{data_folder}"
    result = requests.get(github_url)

    soup = BeautifulSoup(result.text, "html.parser")
    csvfiles = soup.find_all(title=re.compile("\.csv$"))
    xlsxfiles = soup.find_all(title=re.compile("\.xlsx$"))

    filename = []
    for file in csvfiles + xlsxfiles:
        name = file.extract().get_text()
        name = Path(name).resolve().stem
        filename.append(name)
    
    return filename

In [4]:
def get_temperature(city):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/temperatura_sp/{city}.csv"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)
    csv_string = StringIO(result.text)
    temperature_df = pd.read_csv(csv_string, delimiter=";")
    temperature_df["din_medicao"] = pd.to_datetime(temperature_df["din_medicao"])
    temperature_df = temperature_df[temperature_df["id_varmeteo"] == "TEM_MAX"]
    temperature_df = temperature_df[["din_medicao", "val_medicao"]]
    temperature_df = temperature_df[temperature_df["val_medicao"].notnull()]
    temperature_df = temperature_df.sort_values(by="din_medicao")
    temperature_fig = px.line(temperature_df, x='din_medicao', y="val_medicao")
    display(temperature_df)
    display(temperature_fig)

In [6]:
city_temperature = interactive(get_temperature, city=get_data_list("temperatura_sp"))

display(city_temperature)

interactive(children=(Dropdown(description='city', options=('Ariranha', 'Avare', 'Barra Bonita', 'Barra do Tur…

In [7]:
def get_meteorology(city, variable):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/meteorologia_sp/{city}.csv"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)
    csv_string = StringIO(result.text)
    meteorology_df = pd.read_csv(csv_string, delimiter=";")
    meteorology_df["Hora Medicao"] = meteorology_df["Hora Medicao"] / 100
    meteorology_df["Hora Medicao"] = meteorology_df["Data Medicao"].astype(str) + " " + meteorology_df["Hora Medicao"].astype(int).astype(str) + ":00:00"
    meteorology_df["Hora Medicao"] = pd.to_datetime(meteorology_df["Hora Medicao"])
    meteorology_df = meteorology_df[["Hora Medicao", "PRECIPITACAO TOTAL, HORARIO(mm)", "UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)", "VENTO, VELOCIDADE HORARIA(m/s)"]]
    meteorology_df = meteorology_df[meteorology_df["PRECIPITACAO TOTAL, HORARIO(mm)"].notnull()]
    meteorology_df = meteorology_df[meteorology_df["UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)"].notnull()]
    meteorology_df = meteorology_df[meteorology_df["VENTO, VELOCIDADE HORARIA(m/s)"].notnull()]
    meteorology_df = meteorology_df.sort_values(by="Hora Medicao")
    variable_map = {
        "Precipitacao Total": "PRECIPITACAO TOTAL, HORARIO(mm)",
        "Umidade Relativa": "UMIDADE REL. MAX. NA HORA ANT. (AUT)(%)",
        "Velocidade do Vento": "VENTO, VELOCIDADE HORARIA(m/s)"
    }
    variable_name = variable_map[variable]
    meteorology_df = meteorology_df[["Hora Medicao", variable_name]]
    meteorology_fig = px.line(meteorology_df, x="Hora Medicao", y=variable_name)
    display(meteorology_df)
    display(meteorology_fig)

In [8]:
city_meteorology = interactive(get_meteorology, city=get_data_list("meteorologia_sp"), variable=["Precipitacao Total", "Umidade Relativa", "Velocidade do Vento"])

display(city_meteorology)

interactive(children=(Dropdown(description='city', options=('Ariranha', 'Avare', 'Barra Bonita', 'Barra do Tur…

In [19]:
def get_load(utility):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/carga_sp/{utility}.xlsx"
    github_url = github_url.replace(" ", "%20")
    load_df = pd.read_excel(github_url)
    load_df["din_ocorrencia"] = pd.to_datetime(load_df["din_ocorrencia"])
    load_df = load_df[["din_ocorrencia", "val_itemserieoriginal"]]
    load_df = load_df[load_df["val_itemserieoriginal"].notnull()]
    load_df = load_df.sort_values(by="din_ocorrencia")
    load_fig = px.line(load_df, x="din_ocorrencia", y="val_itemserieoriginal")
    display(load_df)
    display(load_fig)

In [20]:
utility_load = interactive(get_load, utility=get_data_list("carga_sp"))

display(utility_load)

interactive(children=(Dropdown(description='utility', options=('cpfl-paulista', 'cpfl-piratininga', 'cpfl-stac…

In [22]:
map_url = "https://www.google.com/maps/d/embed?mid=1kPQFq3OwYmW0bN-WeRcOi97vUW0ARPI&ehbc=2E312F"
IFrame(map_url, width=900, height=650)